# 3. Document Classification Based on BOW


## 1. Preparing the 20 Newsgroups Data and Feature Extraction

 http://scikit-learn.org/0.19/datasets/twenty_newsgroups.html

In [1]:
from sklearn.datasets import fetch_20newsgroups

# Create a list of topics to select from the 20 categories
categories = ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

# Fetch the training dataset
newsgroups_train = fetch_20newsgroups(subset='train',
# Remove hinting parts from the email content - classify purely based on content
                                      remove=('headers', 'footers', 'quotes'),
                                      categories=categories)

# Fetch the test dataset
newsgroups_test = fetch_20newsgroups(subset='test',
                                     remove=('headers', 'footers', 'quotes'),
                                     categories=categories)

print('#Train set size:', len(newsgroups_train.data))
print('#Test set size:', len(newsgroups_test.data))
print('#Selected categories:', newsgroups_train.target_names)
print('#Train labels:', set(newsgroups_train.target))

#Train set size: 2034
#Test set size: 1353
#Selected categories: ['alt.atheism', 'comp.graphics', 'sci.space', 'talk.religion.misc']
#Train labels: {0, 1, 2, 3}


In [2]:
print('#Train set text samples:', newsgroups_train.data[0])
print('#Train set label smaples:', newsgroups_train.target[0])
print('#Test set text samples:', newsgroups_test.data[0])
print('#Test set label smaples:', newsgroups_test.target[0])

#Train set text samples: Hi,

I've noticed that if you only save a model (with all your mapping planes
positioned carefully) to a .3DS file that when you reload it after restarting
3DS, they are given a default position and orientation.  But if you save
to a .PRJ file their positions/orientation are preserved.  Does anyone
know why this information is not stored in the .3DS file?  Nothing is
explicitly said in the manual about saving texture rules in the .PRJ file. 
I'd like to be able to read the texture rule information, does anyone have 
the format for the .PRJ file?

Is the .CEL file format available from somewhere?

Rych
#Train set label smaples: 1
#Test set text samples: TRry the SKywatch project in  Arizona.
#Test set label smaples: 2


In [3]:
X_train = newsgroups_train.data   # Training dataset documents
y_train = newsgroups_train.target # Training dataset labels

X_test = newsgroups_test.data     # Test dataset documents
y_test = newsgroups_test.target   # Test dataset labels

## 2. Document Representation Based on Distributed Representation

### 1) Word2Vec

In [4]:
# Import necessary libraries for Word2Vec and machine learning models
import gensim
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from collections import Counter
from nltk.tokenize import word_tokenize
import nltk

# Download required NLTK resources for tokenization and stopwords
nltk.download('punkt')  # For word tokenization
nltk.download('stopwords')  # For filtering out common stopwords

# Initialize a set of stopwords for English and a stemmer to reduce words to their root form
stop_words = set(stopwords.words('english'))
stemmer = PorterStemmer()  # PorterStemmer is commonly used to reduce words to their stem form

# Data preprocessing function
def preprocess_data(data):
    processed_data = []
    for sentence in data:
        # Tokenize the sentence into words
        tokens = word_tokenize(sentence)
        # Convert to lowercase, remove stopwords and special characters, and apply stemming
        tokens = [stemmer.stem(re.sub(r'\W+', '', word.lower()))
                  for word in tokens
                  if word.lower() not in stop_words and re.sub(r'\W+', '', word)]
        processed_data.append(tokens)  # Add the cleaned tokens to the processed data
    return processed_data

# Step 1: Train the Word2Vec model
# Preprocess the training and testing data using the preprocess_data function
X_train_tokenized = preprocess_data(X_train)
X_test_tokenized = preprocess_data(X_test)

# Flatten the tokenized training data into a single list and calculate word frequency
all_words = [word for sentence in X_train_tokenized for word in sentence]
word_counts = Counter(all_words)  # Count frequency of each word in the training data

# Define a threshold to remove low-frequency words
min_count_threshold = 2  # Words with a frequency of 2 or lower will be removed
# Filter the tokenized training and testing data to keep only frequent words
X_train_tokenized = [[word for word in sentence if word_counts[word] > min_count_threshold] for sentence in X_train_tokenized]
X_test_tokenized = [[word for word in sentence if word_counts[word] > min_count_threshold] for sentence in X_test_tokenized]

# Train the Word2Vec model with the tokenized and filtered training data
# vector_size: Dimensionality of the word vectors
# window: Maximum distance between the current and predicted word
# sg: Use skip-gram (1) instead of CBOW (0)
w2v_model = Word2Vec(sentences=X_train_tokenized, vector_size=100, window=2, min_count=2, sg=1)

# Step 2: Function to generate Word2Vec vectors for each sentence
# Each sentence vector is computed as the average of its word vectors
# The function calculates each sentence's vector by averaging the vectors of the words present in the Word2Vec model. It sums the vectors for the valid words in the sentence and divides by the total number of those words to find the average, resulting in a single vector representation for the sentence.

def get_w2v_vectors(data, model, vector_size=100):
    vectors = []
    for sentence in data:
        # Initialize a zero vector for the sentence
        sentence_vec = np.zeros(vector_size)
        count = 0  # Track the number of words found in the Word2Vec model
        for word in sentence:
            # Check if the word exists in the Word2Vec model
            if word in model.wv.key_to_index:
                # Add the word vector to the sentence vector
                sentence_vec += model.wv[word]
                count += 1
        # If the sentence contains valid words, compute the average of the word vectors
        if count != 0:
            # The sentence vector is the average of the word vectors in the sentence
            sentence_vec /= count
        # Append the resulting sentence vector (average of word vectors) to the list
        vectors.append(sentence_vec)
    return np.array(vectors)

# Generate Word2Vec vectors for both the training and testing data
X_train_w2v = get_w2v_vectors(X_train_tokenized, w2v_model)
X_test_w2v = get_w2v_vectors(X_test_tokenized, w2v_model)

# Step 3: Train machine learning models (Logistic Regression and Random Forest)
# Initialize two different classifiers for comparison
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000),  # Logistic Regression with maximum iterations set to 1000
    'Random Forest': RandomForestClassifier()  # Random Forest Classifier
}

# Dictionary to store model names and their performance metrics
results = {'Model': [], 'Train Accuracy': [], 'Test Accuracy': []}

# Train each model and evaluate accuracy on the training and testing data
for model_name, model in models.items():
    # Fit the model using the Word2Vec vectorized training data and corresponding labels
    model.fit(X_train_w2v, y_train)
    # Calculate accuracy on both the training and testing datasets
    train_acc = model.score(X_train_w2v, y_train)
    test_acc = model.score(X_test_w2v, y_test)

    # Store the results for each model
    results['Model'].append(model_name + " (Word2Vec)")  # Add the model name and the method used (Word2Vec)
    results['Train Accuracy'].append(train_acc)  # Training accuracy
    results['Test Accuracy'].append(test_acc)  # Testing accuracy

[nltk_data] Downloading package punkt to /home/minjoo/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /home/minjoo/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [5]:
print(results)

{'Model': ['Logistic Regression (Word2Vec)', 'Random Forest (Word2Vec)'], 'Train Accuracy': [0.683874139626352, 0.9783677482792527], 'Test Accuracy': [0.6400591278640059, 0.6577974870657798]}
